In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import os
import hopsworks

api_key = os.getenv('HOPSWORKS_API_KEY')  
project_name = os.getenv('HOPSWORKS_PROJECT_NAME')  

# print("API Key:", api_key)
print("Project Name:", project_name)

# Initialize connection to Hopsworks  
project = hopsworks.login(api_key_value=api_key, project=project_name)
print(f"Successfully connected to Hopsworks project: {project_name}")

Project Name: new_york_taxi
2025-03-03 14:09:01,078 INFO: Initializing external client
2025-03-03 14:09:01,079 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 14:09:02,235 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677
Successfully connected to Hopsworks project: new_york_taxi


In [4]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-03 20:00:00+00:00


In [5]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [6]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-03 14:09:03,685 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 14:09:03,690 INFO: Initializing external client
2025-03-03 14:09:03,691 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 14:09:04,420 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677


In [7]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.52s) 


In [8]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,177,1.0,2025-03-03 15:00:00+00:00
1,72,2.0,2025-03-03 15:00:00+00:00
2,16,0.0,2025-03-03 15:00:00+00:00
3,209,4.0,2025-03-03 15:00:00+00:00
4,68,123.0,2025-03-03 15:00:00+00:00
...,...,...,...
748,163,188.0,2025-03-03 20:00:00+00:00
749,208,0.0,2025-03-03 20:00:00+00:00
750,256,2.0,2025-03-03 20:00:00+00:00
751,115,0.0,2025-03-03 20:00:00+00:00


In [9]:
#a.info()

In [10]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=0)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-03 14:09:08,083 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 14:09:08,090 INFO: Initializing external client
2025-03-03 14:09:08,090 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 14:09:08,754 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677
Current UTC time: 2025-03-03 19:09:08.083145+00:00
Next hour: 2025-03-03 19:00:00+00:00
Querying for date range: 2025-03-03 to 2025-03-04
Filtering for hour: 2025-03-03 19:00
Found 251 records


In [11]:
now = datetime.now(timezone.utc)

In [12]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
251,151,37.0,2025-03-03 19:00:00+00:00
252,250,0.0,2025-03-03 19:00:00+00:00
253,92,0.0,2025-03-03 19:00:00+00:00
254,64,0.0,2025-03-03 19:00:00+00:00
255,35,1.0,2025-03-03 19:00:00+00:00
...,...,...,...
497,183,0.0,2025-03-03 19:00:00+00:00
498,82,0.0,2025-03-03 19:00:00+00:00
499,135,0.0,2025-03-03 19:00:00+00:00
500,34,0.0,2025-03-03 19:00:00+00:00


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753 entries, 0 to 752
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  753 non-null    int32                  
 1   predicted_demand    753 non-null    float64                
 2   pickup_hour         753 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 14.8 KB


In [14]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [15]:
dt = current_date.ceil('h')


In [16]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-03 14:09:11,117 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 14:09:11,121 INFO: Initializing external client
2025-03-03 14:09:11,122 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 14:09:11,934 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (53.58s) 


In [17]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.41s) 


,pickup_location_id,predicted_demand,pickup_hour


In [18]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,177,1.0,2025-03-03 15:00:00+00:00
1,72,2.0,2025-03-03 15:00:00+00:00
2,16,0.0,2025-03-03 15:00:00+00:00
3,209,4.0,2025-03-03 15:00:00+00:00
4,68,123.0,2025-03-03 15:00:00+00:00
...,...,...,...
748,163,188.0,2025-03-03 20:00:00+00:00
749,208,0.0,2025-03-03 20:00:00+00:00
750,256,2.0,2025-03-03 20:00:00+00:00
751,115,0.0,2025-03-03 20:00:00+00:00


In [19]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [20]:
predictions = fetch_next_hour_predictions()

2025-03-03 14:10:14,698 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 14:10:14,701 INFO: Initializing external client
2025-03-03 14:10:14,702 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 14:10:15,519 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677
Current UTC time: 2025-03-03 19:10:14.698286+00:00
Next hour: 2025-03-03 20:00:00+00:00
Found 251 records


In [21]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-03-03 14:10:16,875 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 14:10:16,880 INFO: Initializing external client
2025-03-03 14:10:16,880 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 14:10:17,644 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677


In [22]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (53.30s) 


In [23]:
df["pickup_hour"].max()

Timestamp('2025-03-03 20:00:00+0000', tz='Etc/UTC')

In [24]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-03 20:00:00+00:00


In [25]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
502,249,70.0,2025-03-03 20:00:00+00:00
503,178,0.0,2025-03-03 20:00:00+00:00
504,10,2.0,2025-03-03 20:00:00+00:00
505,179,1.0,2025-03-03 20:00:00+00:00
506,219,1.0,2025-03-03 20:00:00+00:00
...,...,...,...
748,163,188.0,2025-03-03 20:00:00+00:00
749,208,0.0,2025-03-03 20:00:00+00:00
750,256,2.0,2025-03-03 20:00:00+00:00
751,115,0.0,2025-03-03 20:00:00+00:00
